# QLD Conservation Status List

This notebook downloads the Qld lists from the [Qld Government Open Data Portal](https://data.qld.gov.au) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/QLD` directory, process the lists and save them to `current-lists`.

## Lists in the ALA Species List tool
* Conservation list: __[dr652](https://lists.ala.org.au/speciesListItem/list/dr652)__ ([dr652 in test](https://lists-test.ala.org.au/speciesListItem/list/dr652))
* Collection: __[dr652](https://collections.ala.org.au/public/show/dr652)__

## Source Data
Queensland Nature Conservation Act 1992

* __[Conservation](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv)__
* __[Species Codes](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv)__

**Metadata Description**

**Conservation:** The list of taxa from the Department of Environment and Science’s WildNet database with their classification codes under the Nature Conservation Act 1992: Extinct (EX), Extinct in the wild (PE), Critically Endangered (CR), Endangered (E), Vulnerable (V), Near threatened (NT, Least concern (C), Special least concern (SL) and International (I). It is published weekly as the Conservation status of Queensland wildlife dataset in the Queensland Government Data portal.

**Metadata URL**
* Qld Species (Open Data Portal) https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife
* Queensland Confidential Species (Open Data Portal) https://www.data.qld.gov.au/dataset/queensland-confidential-species
* Qld Species codes https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife/resource/6344ea93-cadf-4e0c-9ff4-12dfb18d5f14

In [53]:
import datetime

import pandas as pd
import requests
import io
from ftfy import fix_encoding
import urllib.request, json
import certifi
import ssl
import os
import sys
import os.path as path

# top level directory
projectDir = path.abspath(path.join(os.getcwd(),"../../..")) + "/"

sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

sourceDataDir = projectDir + "source-data/QLD/"
statusDir = projectDir + "source-data/status-codes/"
processedDataDir = projectDir + "current-lists/"
state = 'QLD'
monthStr = datetime.datetime.now().strftime('%Y%m')
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"
listurl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv"

testListDruid = "dr652"
prodListDruid = "dr652"

## Download the raw files from data.qld.gov.au
... save locally

In [54]:
# %%script echo skipping # comment this line to download dataset from API

# Status codes
response = requests.get(codesurl)
rtext = fix_encoding(response.text)
speciescodes = pd.read_csv(io.StringIO(rtext))
speciescodes.to_csv(sourceDataDir + "species-status-codes.csv", index=False)

# Conservation List
response = requests.get(listurl)
rtext = fix_encoding(response.text)
conservationlist = pd.read_csv(io.StringIO(rtext))
conservationlist.to_csv(sourceDataDir + "species.csv", index=False)

## Standardise Status Codes
Some minimal changes to some Qld Nature Conservation Act codes so that they are consistent with other states

In [55]:
speciescodes = pd.read_csv(sourceDataDir + "species-status-codes.csv")
ncastatuscodes = speciescodes[speciescodes['Field'] == "NCA_status"][['Code', 'Code_description']]
ncastatuscodes['Code_description'] = ncastatuscodes['Code_description'].str.replace(" wildlife", "")
ncastatuscodes.loc[
    ncastatuscodes['Code_description'] == "Critically endangered", 'Code_description'] = "Critically Endangered"
ncastatuscodes.loc[ncastatuscodes['Code_description'] == "Near threatened", 'Code_description'] = "Near Threatened"
ncastatuscodes

,Code,Code_description
20,C,Least concern
21,CR,Critically Endangered
22,E,Endangered
23,EX,Extinct
24,I,International
25,NT,Near Threatened
26,P,Prohibited
27,PE,Extinct in the wild
28,SL,Special least concern
29,V,Vulnerable


## Conservation List
* Join to the codes to expand the code descriptions.
* Change the field names to `sourceStatus` and `status` as required by the ALA's conservation list processing.
* Remove **Least concern**, **Special least concern** and **no status**
* Expand the endemicity and epbc status codes
* Map sourceStatus to status

In [56]:
conservationlist = pd.read_csv(sourceDataDir + "species.csv")
conservationlist = pd.merge(conservationlist,ncastatuscodes,left_on=['NCA_status'],right_on=['Code'],how="left")

conservationlist = conservationlist.rename(columns=
{   'Scientific_name':'scientificName',
    'Common_name': 'vernacularName',
    'Taxon_author':'scientificNameAuthorship',
    'Family': 'family',
    'NCA_status':'sourceStatus',
    'Code_description':'status',
    'Taxon_Id':'WildNetTaxonID'
})

conservationlist['taxonID'] = 'https://apps.des.qld.gov.au/species-search/details/?id=' + conservationlist['WildNetTaxonID'].astype(str)

# remove unwanted rows
conservationlist = conservationlist[(conservationlist['status'].notna())]
conservationlist = conservationlist[~conservationlist['status'].str.contains('Special least concern', case=False)]
conservationlist = conservationlist[~conservationlist['status'].str.contains('Least concern', case=False)]
# remove unwanted cols
conservationlist = conservationlist.loc[:, ['scientificName', 'vernacularName', 'family', 'WildNetTaxonID','taxonID','status', 'sourceStatus']]
conservationlist.groupby('status')['scientificName'].count()

status
Critically Endangered    146
Endangered               311
Extinct                    6
Extinct in the wild       26
Near Threatened          280
Vulnerable               579
Name: scientificName, dtype: int64

## Manual Additions

In [57]:
# recommended manual adds
adds = conservationlist[conservationlist['scientificName'].isin(['Cacatua leadbeateri leadbeateri','Eclectus polychloros macgillivrayi'])]
adds.loc[adds['scientificName'] == "Cacatua leadbeateri leadbeateri",'scientificName'] = "Cacatua leadbeateri"
adds.loc[adds['scientificName'] == "Eclectus polychloros macgillivrayi",'scientificName'] = "Eclectus polychloros"
conservationlist = pd.concat([conservationlist,adds])
conservationlist.loc[len(conservationlist)] = ["Eclectus roratus","","Psittaculidae","1168","https://apps.des.qld.gov.au/species-search/details/?id=1168","Vulnerable","V"]
conservationlist.groupby('status')['scientificName'].count()

status
Critically Endangered    146
Endangered               312
Extinct                    6
Extinct in the wild       26
Near Threatened          280
Vulnerable               581
Name: scientificName, dtype: int64

In [58]:
# write to file
conservationlist.to_csv(processedDataDir + 'conservation-lists/QLD-' + prodListDruid + '-conservation.csv',encoding="UTF-8",index=False)

## Manual List check
### Instructions
1. Load the list above into the lists-test tool for this data resource
2. Unskip the below code and Run the reports below to compare to production. Send the changelog.csv to check. Correct any issues.
3. Save the production list into the historical lists directory by uncommenting the code section below.
4. Load the list from current lists into production

In [59]:
ltype = "C"
changeDir = "Monitoring/Change-logs/"
filename = "QLD-conservation.csv"
changelist = lf.get_changelist(testListDruid, prodListDruid, ltype)
changelist.to_csv(projectDir + changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

,name,scientificName,commonName,status,listUpdate,status_old
694,Acacia parvifoliolata,Acacia parvifoliolata,None,Endangered,added,NaN
674,Acacia pedleyi,Acacia pedleyi,None,Endangered,status change,Vulnerable
1009,Actephila bella,Actephila bella,None,Endangered,status change,Vulnerable
1323,Amomum queenslandicum,Amomum queenslandicum,None,Critically Endangered,status change,Vulnerable
375,Argophyllum curtum,Argophyllum curtum,None,Near Threatened,added,NaN
...,...,...,...,...,...,...
645,Romnalda ophiopogonoides,Romnalda ophiopogonoides,None,Endangered,status change,Vulnerable
858,Sannantha brachypoda,Sannantha brachypoda,None,Near Threatened,status change,Vulnerable
1282,Symplocos crassiramifera,Symplocos crassiramifera,None,Endangered,status change,Vulnerable
413,Tetramolopium ornans,Tetramolopium,None,Vulnerable,added,NaN


In [60]:
# Download historical lists from Production
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + prodListDruid + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)